In [18]:
#importing necessary libraries
import pandas as pd
import string
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
import re
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
import torch
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

In [19]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
train_data = pd.read_csv("/content/olid-training.csv")
train_data.shape

(13193, 5)

In [29]:
#displaying the tweets
t_tweets = train_data["tweet"]
print(t_tweets)

0        @USER She should ask a few native Americans wh...
1        @USER @USER Go home you’re drunk!!! @USER #MAG...
2        Amazon is investigating Chinese employees who ...
3        @USER Someone should'veTaken" this piece of sh...
4        @USER @USER Obama wanted liberals &amp; illega...
                               ...                        
13188    @USER every antifa member is just several midg...
13189      @USER Berkeley Antifa do not agree with you URL
13190          @USER You are a great model for inspiration
13191    @USER Chris, Chris, Chris.... Are you forgetti...
13192    @USER Advocate for gun control while breaking ...
Name: tweet, Length: 13193, dtype: object


In [30]:
def clean_tweets(df):
    punctuations = string.punctuation
    stop_words = set(stopwords.words('english'))

    # Remove mentions, URLs, and special characters
    df.loc[:, 'tweet'] = df.tweet.str.replace('@USER', '') \
                                    .str.replace('URL', '') \
                                    .str.replace('&amp', 'and') \
                                    .str.replace('&lt','') \
                                    .str.replace('&gt','') \
                                    .str.replace('\d+','') \
                                    .str.lower()

    # Remove punctuations
    for punctuation in punctuations:
        df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')

    # Remove emojis
    df.loc[:, 'tweet'] = df.tweet.apply(lambda x: re.sub(r'[^\x00-\x7F]+', '', x))


    # Trim leading and trailing whitespaces
    df.loc[:, 'tweet'] = df.tweet.str.strip()

    return df

In [31]:
clean_tweets(train_data)

<ipython-input-30-8f201304bd0e>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  .str.replace('\d+','') \
<ipython-input-30-8f201304bd0e>:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:, 'tweet'] = df.tweet.str.replace(punctuation, '')


,id,tweet,subtask_a,subtask_b,subtask_c
0,86426,she should ask a few native americans what the...,OFF,UNT,NaN
1,90194,go home youre drunk maga trump,OFF,TIN,IND
2,16820,amazon is investigating chinese employees who ...,NOT,NaN,NaN
3,62688,someone shouldvetaken this piece of shit to a ...,OFF,UNT,NaN
4,43605,obama wanted liberals and illegals to move int...,NOT,NaN,NaN
...,...,...,...,...,...
13188,94475,every antifa member is just several midget gyp...,NOT,NaN,NaN
13189,34150,berkeley antifa do not agree with you,NOT,NaN,NaN
13190,83400,you are a great model for inspiration,NOT,NaN,NaN
13191,84081,chris chris chris are you forgetting the mantr...,OFF,UNT,NaN


In [32]:
from sklearn.utils import resample

# Separate majority and minority classes
majority_class = train_data[train_data['subtask_a'] == 'NOT']
minority_class = train_data[train_data['subtask_a'] == 'OFF']

# Downsample majority class
majority_downsampled = resample(majority_class,
                                replace=False,  # Sample without replacement
                                n_samples=len(minority_class),  # Match number in minority class
                                random_state=42)  # Ensure reproducibility

# Combine minority class with downsampled majority class
train_data = pd.concat([majority_downsampled, minority_class])

# Shuffle the downsampled data to avoid any bias in the order
train_data = train_data.sample(frac=1, random_state=42).reset_index(drop=True)


In [33]:
train_data['tweet']

0                 amazing all the projection she is doing
1       you cant tell us conservatives that  didnt con...
2       this is not possible california has implemente...
3       did you find out where senator collins is slee...
4       and it shouldnt matter if they are democrats o...
                              ...                        
8773                              she is a dog with fleas
8774    so we should believe this from  years ago when...
8775    i met mark and he is amazing he is just as sas...
8776    our  usa ww soldiers were all antifa antifasci...
8777    i am absolutely furious with my fellow conserv...
Name: tweet, Length: 8778, dtype: object

Tokenization is the process of breaking down a text into smaller units called tokens. These tokens can be words, subwords, characters, or any other meaningful units depending on the specific tokenization strategy used.

In [34]:
#TOKENIZATION USING BERT
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer & model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)  # Adjust num_labels based on your classification task

def tokenize_tweets_with_bert(tweets):
    tokenized_tweets = [tokenizer.tokenize(tweet) for tweet in tweets]
    return tokenized_tweets

tweets = train_data['tweet']
tokenized_tweets = tokenize_tweets_with_bert(tweets)

# Printing the first few tokenized tweets for verification
for i in range(5):  # Print the first 5 tokenized tweets
    print(f"Tweet {i + 1}: {tokenized_tweets[i]}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Tweet 1: ['amazing', 'all', 'the', 'projection', 'she', 'is', 'doing']
Tweet 2: ['you', 'can', '##t', 'tell', 'us', 'conservatives', 'that', 'didn', '##t', 'con', '##ni', '##ve', 'with', 'to', 'keep', 'all', 'of', 'this', 'under', 'wrap', 'till', 'time', 'for', 'final', 'pot', '##us', 'voted', '##ir', '##ty', '##pol', '##itic', '##s']
Tweet 3: ['this', 'is', 'not', 'possible', 'california', 'has', 'implemented', 'strict', 'gun', 'control', 'only', 'law', 'enforcement', 'and', 'those', 'fortunate', 'enough', 'to', 'get', 'a', 'permit', 'can', 'carry', 'in', 'public', 'assault', 'weapons', 'are', 'banned']
Tweet 4: ['did', 'you', 'find', 'out', 'where', 'senator', 'collins', 'is', 'sleeping', 'let', 'us', 'know', 'so', 'we', 'can', 'send', 'some', 'anti', '##fa', 'goo', '##ns', 'to', 'her', 'house', 'to', 'straighten', 'her', 'out']
Tweet 5: ['and', 'it', 'shouldn', '##t', 'matter', 'if', 'they', 'are', 'democrats', 'or', 'conservatives', 'they', 'all', 'do', 'their', 'jobs', 'they', 'do

Some words have been split into subwords, indicated by the '##' prefix. For example, 'anti' and 'pal' have been split into 'anti##' and 'pal##' respectively, to maintain subword information.

In [35]:
vocab = np.unique(np.concatenate(tokenized_tweets))
vocab_size = len(vocab)

print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 11870


In [36]:
# Convert tokens to input IDs and create attention masks
input_ids = []
attention_masks = []

for tweet in tokenized_tweets:
    encoded_tweet = tokenizer.encode_plus(
        tweet,                      # Sentence to encode
        add_special_tokens=True,    # Add [CLS] and [SEP]
        max_length=128,  # Pad & truncate all sentences
        padding='max_length',
       # pad_to_max_length=True,
        return_attention_mask=True  # Construct attention masks
    )
    input_ids.append(encoded_tweet['input_ids'])
    attention_masks.append(encoded_tweet['attention_mask'])

By converting the tokenized tweets into input IDs and creating attention masks, the data is formatted in a way that can be easily processed by deep learning models, particularly those built using frameworks like TensorFlow or PyTorch.

In [37]:
train_data['labels'] = train_data['subtask_a'].map({"OFF": 0, "NOT": 1})
# Convert labels to tensors
labels = torch.tensor(train_data['labels'] )
len(labels)

8778

In [38]:
# Convert lists to tensors
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

In [39]:
#Splitting data into training & validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, test_size=0.1, random_state=42)
train_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids, test_size=0.1, random_state=42)


In [40]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=128)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=128)

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [42]:
import torch
from torch.nn.utils.rnn import pad_sequence

bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.to(device)
def generate_embeddings(data_loader, max_length):
    embeddings = []
    labels = []

    for batch in tqdm(data_loader, desc="Generating Embeddings"):
        input_ids_batch, attention_masks_batch, labels_batch = batch
        input_ids_batch, attention_masks_batch, labels_batch = input_ids_batch.to(device), attention_masks_batch.to(device), labels_batch.to(device)

        with torch.no_grad():
            outputs = bert_model(input_ids=input_ids_batch, attention_mask=attention_masks_batch)
            pooled_output = outputs.pooler_output  # pooled output for classification tasks

        embeddings.append(torch.tensor(pooled_output.cpu().numpy()))  # Converting NumPy array to Tensor
        labels.append(torch.tensor(labels_batch.cpu().numpy()))  # Converting NumPy array to Tensor

    # Padding the embeddings to get uniform length
    padded_embeddings = torch.nn.utils.rnn.pad_sequence(embeddings, batch_first=True, padding_value=0)
    padded_labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0) #setting padding value to 0 so that it matches the length of longest seq in batch

    return padded_embeddings, padded_labels

# Define the maximum length for padding
max_length = 128

# Generate padded embeddings for training and validation sets
train_embeddings, train_labels = generate_embeddings(train_dataloader, max_length)
val_embeddings, val_labels = generate_embeddings(val_dataloader, max_length)


Generating Embeddings: 100%|██████████| 7/7 [00:05<00:00,  1.26it/s]


In [43]:
import tensorflow as tf
# Convert PyTorch tensors to NumPy arrays
train_embeddings_np = train_embeddings.numpy()
train_labels_np = train_labels.numpy()
val_embeddings_np = val_embeddings.numpy()
val_labels_np = val_labels.numpy()

# Convert NumPy arrays to TensorFlow tensors
train_embeddings_tf = tf.convert_to_tensor(train_embeddings_np, dtype=tf.float32)
train_labels_tf = tf.convert_to_tensor(train_labels_np, dtype=tf.float32)
val_embeddings_tf = tf.convert_to_tensor(val_embeddings_np, dtype=tf.float32)
val_labels_tf = tf.convert_to_tensor(val_labels_np, dtype=tf.float32)

In [44]:
# Check the shape of the training embeddings
print("Shape of train_embeddings:", train_embeddings_tf.shape)

# Check the shape of the validation embeddings
print("Shape of val_embeddings:", val_embeddings_tf.shape)

Shape of train_embeddings: (62, 128, 768)
Shape of val_embeddings: (7, 128, 768)


In [45]:
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [52]:
latent_dim = 50 #number of dimensions in the hidden state vector that the LSTM cell maintains during computation.
embedding_dim = 768
vocab_size = 11870
max_length = 128
# Encoder
encoder_inputs = Input(shape=(max_length, embedding_dim))  # Assuming you have already defined `embedding_dim` and None is for variable sequence length
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs) #state h - hidden state, state c -cell state
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(max_length,))
dec_emb_layer = Embedding(vocab_size, embedding_dim, input_length=max_length)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(1, activation='sigmoid')
decoder_outputs = decoder_dense(decoder_outputs)
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [53]:
# Train the model
hist = model.fit([train_embeddings_tf, train_embeddings_tf], train_labels_tf, epochs=10, batch_size=32, validation_data=([val_embeddings_tf, val_embeddings_tf], val_labels_tf))

ValueError: Data cardinality is ambiguous:
  x sizes: 7936, 7936
  y sizes: 62
Make sure all arrays contain the same number of samples.

In [ ]:
'''import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the CNN model architecture
embedding_dim = 300
num_filters = 200
filter_sizes = [7,8,9]  # Different filter sizes for convolution
num_classes = 2  # Offensive or not offensive

# Define the CNN layers
embedding_layer = nn.Embedding(len(tokenizer.vocab), embedding_dim)
conv_layers = nn.ModuleList([
    nn.Conv2d(1, num_filters, (k, embedding_dim)) for k in filter_sizes
])
fc_layer = nn.Linear(num_filters * len(filter_sizes), num_classes)

# Move layers to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
embedding_layer.to(device)
conv_layers.to(device)
fc_layer.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(list(embedding_layer.parameters()) + list(conv_layers.parameters()) + list(fc_layer.parameters()), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    embedding_layer.train()
    for conv_layer in conv_layers:
        conv_layer.train()
    fc_layer.train()

    running_loss = 0.0
    total_correct = 0

    for inputs, masks, labels in train_dataloader:
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        # Forward pass
        embedded = embedding_layer(inputs)  # [batch_size, max_len, embedding_dim]
        embedded = embedded.unsqueeze(1)  # [batch_size, 1, max_len, embedding_dim]

        conv_outputs = []
        for conv_layer in conv_layers:
            conv_output = torch.relu(conv_layer(embedded))  # [batch_size, num_filters, max_len - filter_size + 1, 1]
            conv_output = torch.squeeze(conv_output, -1)  # [batch_size, num_filters, max_len - filter_size + 1]
            pooled_output, _ = torch.max(conv_output, -1)  # [batch_size, num_filters]
            conv_outputs.append(pooled_output)

        combined_output = torch.cat(conv_outputs, dim=1)  # [batch_size, num_filters * len(filter_sizes)]

        logits = fc_layer(combined_output)  # [batch_size, num_classes]

        # Calculate loss
        loss = criterion(logits, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(logits, 1)
        total_correct += (predicted == labels).sum().item()

    # Print statistics
    epoch_loss = running_loss / len(train_dataloader)
    epoch_accuracy = total_correct / len(train_data)
    print(f"Epoch {epoch+1}, Loss: {epoch_loss}, Accuracy: {epoch_accuracy}")

# Evaluation loop (similar to the previous example)
# Validation phase
    embedding_layer.eval()
    for conv_layer in conv_layers:
        conv_layer.eval()
    fc_layer.eval()

    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, masks, labels in val_dataloader:
            inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

            # Forward pass
            embedded = embedding_layer(inputs)  # [batch_size, max_len, embedding_dim]
            embedded = embedded.unsqueeze(1)  # [batch_size, 1, max_len, embedding_dim]

            conv_outputs = []
            for conv_layer in conv_layers:
                conv_output = torch.relu(conv_layer(embedded))  # [batch_size, num_filters, max_len - filter_size + 1, 1]
                conv_output = torch.squeeze(conv_output, -1)  # [batch_size, num_filters, max_len - filter_size + 1]
                pooled_output, _ = torch.max(conv_output, -1)  # [batch_size, num_filters]
                conv_outputs.append(pooled_output)

            combined_output = torch.cat(conv_outputs, dim=1)  # [batch_size, num_filters * len(filter_sizes)]

            logits = fc_layer(combined_output)  # [batch_size, num_classes]

            # Calculate validation accuracy
            _, predicted = torch.max(logits, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_accuracy = val_correct / val_total
    print(f"Validation Accuracy: {val_accuracy}")'''

Epoch 1, Loss: 0.778577164296181, Accuracy: 0.5707070707070707
Validation Accuracy: 0.6102272727272727
Epoch 2, Loss: 0.30220531648205173, Accuracy: 0.9223484848484849
Validation Accuracy: 0.6102272727272727
Epoch 3, Loss: 0.12000936269760132, Accuracy: 0.9746212121212121
Validation Accuracy: 0.6272727272727273
Epoch 4, Loss: 0.06568789563231892, Accuracy: 0.9851010101010101
Validation Accuracy: 0.6329545454545454
Epoch 5, Loss: 0.044331193330787846, Accuracy: 0.9900252525252525
Validation Accuracy: 0.6363636363636364
Epoch 6, Loss: 0.03593950021651483, Accuracy: 0.9921717171717171
Validation Accuracy: 0.6465909090909091
Epoch 7, Loss: 0.025574138320441687, Accuracy: 0.9952020202020202
Validation Accuracy: 0.6363636363636364
Epoch 8, Loss: 0.023717379783310236, Accuracy: 0.9964646464646465
Validation Accuracy: 0.6522727272727272
Epoch 9, Loss: 0.02181021054275334, Accuracy: 0.9967171717171717
Validation Accuracy: 0.6375
Epoch 10, Loss: 0.018624088986055744, Accuracy: 0.9968434343434344